# Hidden Markov Model project
## Machine Learning Fundamentals

|First name|Last name|Master program|Contribution|
|----------|---------|--------------|-------------|
|Hoang Nam|Nguyen|IMLEX|33.3%|
|Chihiro|Tone|IMLEX|33.3%|
|Uzairu|Abubakar|IMLEX|33.3%|

# WARNING : you need version 0.2.6 of hmmlearn

In [3]:
!pip install --force-reinstall hmmlearn==0.2.6
import hmmlearn
from hmmlearn import hmm

Defaulting to user installation because normal site-packages is not writeable
  Using cached hmmlearn-0.2.6-cp310-cp310-linux_x86_64.whl
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.1.0
    Uninstalling threadpoolctl-3.1.0:
      Successfully uninstalled threadpoolctl-3.1.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successf

In [4]:
!pip install seaborn
!pip install nltk
!pip show hmmlearn  # check that the version installed is 0.2.6

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Name: hmmlearn
Version: 0.2.6
Summary: Hidden Markov Models in Python with scikit-learn like API
Home-page: https://github.com/hmmlearn/hmmlearn
Author: 
Author-email: 
License: new BSD
Location: /home/tc11802x/.local/lib/python3.10/site-packages
Requires: numpy, scikit-learn, scipy
Required-by: 


# WARNING : check that the hmmlearn installed is version 0.2.6

# imports

In [5]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt  # show graph

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

## In this notebook we will look at the NER dataset and use it to understand HMM and also construct a POS tagger at the same time.

### Data Description:
#### sentence: this column donates to which sentence the word belongs
#### Word: the word in the sentence
#### POS: Associated POS tag for the word

## Load the data

Otherwise, if you are working locally or if you just uploaded the dataset for this session on the drive, you can use the following to import it.

In [6]:
_data = pd.read_csv("./data/NER dataset.csv", encoding='latin1')
_data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [7]:
_data = _data.fillna(method="ffill")
_data = _data.rename(columns={'Sentence #': 'sentence'})
_data.head(10)

,sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


# Data pre-processing
If you want to do some pre-processing (lowercase any words, remove stop words, replace numbers/names by a unique NUM/NAME token, etc.) you can do it here in the pipeline.

Note : you could create a new dataset `data_pre_precessed = pre_process(data)` to keep both version and compare the effect of you pre-processing.

In [8]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def pre_processing(src):
    processed_data = src.copy(deep=True)
    processed_data.Word = processed_data.Word.str.lower()
    processed_data.Word = processed_data.Word.apply(lambda x: re.sub(r'\d+(\.*,*/*-*\d+)?', 'NUM', x))
    # stop_words = set(stopwords.words('english'))
    # processed_data['Word'] = processed_data['Word'].apply(lambda x: x if x not in stop_words else '')
    processed_data = processed_data.reset_index(drop=True)
    return processed_data

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tc11802x/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
data = pre_processing(_data)
data.head(10)

,sentence,Word,POS,Tag
0,Sentence: 1,thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,london,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


First let's collect the unique words and the unique POS tags in the dataset, we will use this to construct the HMM later

In [10]:
tags = list(set(data.POS.values))  # Unique POS tags in the dataset
words = list(set(data.Word.values))  # Unique words in the dataset
len(tags), len(words)

(42, 29876)

### We have 42 different tags and 35,178 different words, so the HMM that we construct will have the following properties
- The hidden states of the this HMM will correspond to the POS tags, so we will have 42 hidden states.
- The Observations for this HMM will correspond to the sentences and their words.

#### Before constructing the HMM, we will split the data into train and test.

In [11]:
y = data.POS
X = data.drop('POS', axis=1)

gs = GroupShuffleSplit(n_splits=2, test_size=.33, random_state=42)
train_ix, test_ix = next(gs.split(X, y, groups=data['sentence']))

data_train = data.loc[train_ix]
data_test = data.loc[test_ix]

In [12]:
data_train.head(5)

,sentence,Word,POS,Tag
24,Sentence: 2,families,NNS,O
25,Sentence: 2,of,IN,O
26,Sentence: 2,soldiers,NNS,O
27,Sentence: 2,killed,VBN,O
28,Sentence: 2,in,IN,O


In [13]:
data_test.head(30)

,sentence,Word,POS,Tag
0,Sentence: 1,thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,london,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


Now lets encode the POS and Words to be used to generate the HMM.

In [14]:
dfupdate = data_train.sample(frac=.15, replace=False, random_state=42)
dfupdate.Word = 'UNKNOWN' # hide 15% of words
data_train.update(dfupdate)
words = list(set(data_train.Word.values))
# Convert words and tags into numbers
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}
id2tag = {i: t for i, t in enumerate(tags)}
len(tags), len(words) # variation of the words should be decreased

(42, 23720)

In your theory classes you might have seen that the Hidden Markov Models can be learned by using the Baum-Welch algorithm by just using the observations.
Although we can learn the Hidden States (POS tags) using Baum-Welch algorithm,We cannot map them back the states (words) to the POS tag. So for this exercise we will skip using the BW algorithm and directly create the HMM.

For creating the HMM we should build the following three parameters. 
- `startprob_`
- `transmat_`
- `emissionprob_`

To construct the above mentioned paramters let's first create some useful matrices that will assist us in creating the above three parameters

In [15]:
count_tags = dict(data_train.POS.value_counts())  # Total number of POS tags in the dataset
# Now let's create the tags to words count
count_tags_to_words = data_train.groupby(['POS']).apply(
    lambda grp: grp.groupby('Word')['POS'].count().to_dict()).to_dict()
# We shall also collect the counts for the first tags in the sentence
count_init_tags = dict(data_train.groupby('sentence').first().POS.value_counts())

# Create a mapping that stores the frequency of transitions in tags to it's next tags
count_tags_to_next_tags = np.zeros((len(tags), len(tags)), dtype=int)
sentences = list(data_train.sentence)
pos = list(data_train.POS)

for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

100%|██████████| 702936/702936 [00:00<00:00, 918873.04it/s]


Now Let's build the parameter matrices 

In [16]:
startprob = np.zeros((len(tags),))
transmat = np.zeros((len(tags), len(tags)))
emissionprob = np.zeros((len(tags), len(words)))
num_sentences = sum(count_init_tags.values())
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)
for tag, tagid in tqdm(tag2id.items(), position=0, leave=True):
    floatCountTag = float(count_tags.get(tag, 0))
    startprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        emissionprob[tagid][wordid] = count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    for tag2, tagid2 in tag2id.items():
        transmat[tagid][tagid2] = count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

100%|██████████| 42/42 [00:01<00:00, 40.76it/s]


## Task 1: Similar to how we built the hidden state transition probability matrix as shown above, you will built the transition probability between the words. With this matrix write a function that can calculate the log likelihood given a sentence.

In [17]:
# Calculate the word transition matrix, this matrix will be of shape words x words
count_words_to_next_words = np.zeros((len(words), len(words)), dtype=int)
words_ = list(data_train.Word)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevwordid = word2id[words_[i - 1]]
        nextwordid = word2id[words_[i]]
        count_words_to_next_words[prevwordid][nextwordid] += 1

word_transition_matrix = np.zeros((len(words), len(words)))
sum_words_to_next_words = np.sum(count_words_to_next_words, axis=1)
for word, wordid in tqdm(word2id.items(), position=0, leave=True):
    if sum_words_to_next_words[wordid] == 0:
        continue
    for word2, wordid2 in word2id.items():
        word_transition_matrix[wordid][wordid2] = count_words_to_next_words[wordid][wordid2] / sum_words_to_next_words[wordid]

100%|██████████| 23720/23720 [06:39<00:00, 59.36it/s]


In [18]:
def sentence2idlist(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\d+(\.*,*/*-*\d+)?', 'NUM', sentence)
    return [word2id[w] for w in sentence.split(' ')]

In [19]:
def calculate_log_likelihood(sentence, word_transition_matrix) -> float:
    """
    Write a function given a sentence and transition_matrix will return the loglikehood of the sentence
    """
    l = 0
    for i, wordid in enumerate(sentence):
        if i == 0:
            prevwordid = wordid
            continue
        l += np.log(word_transition_matrix[prevwordid][wordid])
        prevwordid = wordid
    return l

In [20]:
# calculate_log_likelihood(sentence2idlist("This is a test sentence"), word_transition_matrix)
# calculate_log_likelihood(sentence2idlist("Who are you ?"), word_transition_matrix)
# calculate_log_likelihood(sentence2idlist("This is a protest about how the new law is not in the interest of the people"), word_transition_matrix)
# calculate_log_likelihood(sentence2idlist("You are not me"), word_transition_matrix)
calculate_log_likelihood(sentence2idlist("do you expect to be happy to work late"), word_transition_matrix)

-35.11892547319604

#### Now we will continue to constructing the HMM.

We will use the hmmlearn implementation to initialize the HMM Model

In [21]:
model = hmm.MultinomialHMM(n_components=len(tags), algorithm='viterbi', random_state=42)
model.startprob_ = startprob # pi
model.transmat_ = transmat # A
model.emissionprob_ = emissionprob # B

#### Before using the HMM to predict the POS tags, we have to fix the training set as some of the words and tags in the test data might not appear in the training data so we collect this data to use it later.

In [22]:
data_test.loc[~data_test['Word'].isin(words), 'Word'] = 'UNKNOWN'
word_test = list(data_test.Word)
samples = []
for i, val in enumerate(word_test):
    samples.append([word2id[val]])

# TODO use pandas solution
lengths = []
count = 0
sentences = list(data_test.sentence)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        count += 1
    elif i > 0:
        lengths.append(count)
        count = 1
    else:
        count = 1

100%|██████████| 345639/345639 [00:00<00:00, 1139412.11it/s]


Now that we have the HMM ready lets predict the best path from them.

In [23]:
pos_predict = model.predict(samples, lengths)
pos_predict

array([ 5, 20,  5, ..., 26, 14, 38], dtype=int32)

The hmmlearn predict function will give the best probable path for the given sentence using the Viterbi algorithm.

## Task 2: Using the model parameters (startprob_, transmat_, emissionprob_) write the viterbi algorithm from scratch to calculate the best probable path and compare it with the hmmlearn implementation.

Now before using these matrices 

In [24]:
# def Viterbi(pi: np.array, a: np.array, b: np.array, obs: List) -> np.array():
def Viterbi(pi, a, b, obs):
    """
    Write the viterbi algorithm from scratch to find the best probable path
    attr:
        pi: initial probabilities
        a: transition probabilities
        b: emission probabilities
        obs: list of observations
    return:
        array of the indices of the best hidden states
    """
    n_state = len(pi)
    n_transit = len(obs)
    delta = np.zeros((n_transit, n_state))
    phi = np.zeros((n_transit, n_state), dtype=int)

    # Init
    for i in range(n_state):
        delta[0][i] = pi[i] * b[i][obs[0]]
        phi[0][i] = 0

    # Induction
    for t in range(n_transit-1):
        for j in range(n_state):
            delta[t+1][j] = np.max(np.array([delta[t][i] * a[i][j] for i in range(n_state)])) * b[j][obs[t+1]]
            phi[t+1][j] = np.argmax(np.array([delta[t][i] * a[i][j] for i in range(n_state)]))
    
    # Final
    p_ml = np.max(delta[n_transit-1])
    q_ml = np.zeros(n_transit, dtype=int)
    q_ml[n_transit-1] = np.argmax(delta[n_transit-1])
    for t in np.arange(n_transit-2, -1, -1, dtype=int):
        q_ml[t] = phi[t+1][q_ml[t+1]]
    
    return q_ml, p_ml

In [25]:
test_sentence = "Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country ."
q_ml, p_ml = Viterbi(pi=startprob, a=transmat, b=emissionprob, obs=sentence2idlist(test_sentence))
print([id2tag[q] for q in q_ml])
print(q_ml)
print(pos_predict[0:24])

['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']
[ 5 20  5  2  1 20 26 18 36 39 14 20 26  0 36 39 14 20 28  5 20 39 14 38]
[ 5 20  5  2  1 20 26 18 36 39 14 20 26  0 36 39 14 20 28  5 20 39 14 38]


### Task 3: Let's try to form our own HMM
In this task you will try to formulate your own HMM. Image a toy example that you think that closely relates to a Hidden Markov Model.

Steps:
 1. Define your hidden states
 2. Define your observable states
 3. Randomly generate your observations

Below is an example to demonstrate:

-In this toy HMM example, we have two hidden states 'healthy' and 'sick' these states relate to the state of a pet. In this example we cannot exactly know the situation of the pet if it is 'healthy' or 'sick'

-The observable states in this formulation is the what our pet is doing, whether it is sleeping, eating or pooping. We ideally want to determine if the pet is sick or not using these observable states


```python
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observations = []
for i in range(100):
  observations.append(random.choice(observable_states))
```

TASK 3: Now try to formulate your HMM here.

In [42]:
import random
random.seed(42)

hidden_states = ['hot', 'mild', 'cold']
observable_states = ['dress', 't-shirt', 'coat']

# make dataset
N = 100
states = []
observations = []
for i in range(N):
    states.append(random.choice(hidden_states))

for s in states:
    rnd = random.random()
    if s == 'hot':
        if rnd < 0.60:
            observations.append(random.choice(['t-shirt']))
        else:
            observations.append(random.choice(['dress']))
    elif s == 'mild':
        if rnd < 0.60:
            observations.append(random.choice(['dress', 't-shirt']))
        else:
            observations.append(random.choice(['coat']))
    else:
        if rnd < 0.10:
            observations.append(random.choice(['dress', 't-shirt']))
        else:
            observations.append(random.choice(['coat']))


In [43]:
obs2id = {o: i for i,o in enumerate(observable_states)}
sta2id = {s: i for i,s in enumerate(hidden_states)}
id2sta = {i: s for i,s in enumerate(hidden_states)}

In [44]:
startprobs = np.zeros(len(hidden_states))
startprobs[sta2id[states[0]]] = 1

transitprobmat = np.zeros((len(hidden_states),len(hidden_states)))
for t in range(N):
    if t == 0:
        prevstate = sta2id[states[t]]
        continue
    transitprobmat[prevstate][sta2id[states[t]]] += 1
    prevstate = sta2id[states[t]]
for i in range(len(hidden_states)):
    if np.sum(transitprobmat[i]) != 0:
        transitprobmat[i] /= np.sum(transitprobmat[i])

emissionprobmat = np.zeros((len(hidden_states),len(observable_states)))
for t in range(N):
    emissionprobmat[sta2id[states[t]]][obs2id[observations[t]]] += 1
for i in range(len(observable_states)):
    if np.sum(emissionprobmat[i]) != 0:
        emissionprobmat[i] /= np.sum(emissionprobmat[i])

print(startprobs)
print(transitprobmat)
print(emissionprobmat)

[0. 0. 1.]
[[0.36842105 0.23684211 0.39473684]
 [0.44444444 0.2962963  0.25925926]
 [0.35294118 0.32352941 0.32352941]]
[[0.34210526 0.65789474 0.        ]
 [0.42857143 0.25       0.32142857]
 [0.05882353 0.08823529 0.85294118]]


In [45]:
ourmodel = hmm.MultinomialHMM(n_components=len(hidden_states), algorithm='viterbi', random_state=42)
ourmodel.startprob_ = startprobs # pi
ourmodel.transmat_ = transitprobmat # A
ourmodel.emissionprob_ = emissionprobmat # B

In [46]:
test_obs = [random.choice(observable_states) for i in range(10)]
print(test_obs)
test_obs_id = [obs2id[obs] for obs in test_obs]
pos_predict = ourmodel.predict([test_obs_id], 1)
print([id2sta[p] for p in pos_predict])

['t-shirt', 'dress', 't-shirt', 'coat', 'coat', 't-shirt', 'coat', 'coat', 'coat', 'dress']
['cold', 'mild', 'hot', 'cold', 'cold', 'hot', 'cold', 'cold', 'cold', 'mild']


Even tough we have generated the data randomly, for the learning purposes, let's try to learn an HMM from this data. For this we have to construct the Baum-Welch algorithm from scratch. Below is the skeleton of the Baum-Welch learning algorithm.

## TASK 4: Complete the forward and backward probs functions in the Baum-Welch algorithm and try it with your formulated HMM.

In [66]:
import numpy as np


def baum_welch(observations, observations_vocab, n_hidden_states):
    """
    Baum-Welch algorithm for estimating the HMM parameters
    :param observations: observations
    :param observations_vocab: observations vocabulary
    :param n_hidden_states: number of hidden states to estimate
    :return: a, b (transition matrix and emission matrix)
    """

    def forward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        forward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined alpha_
        """
        a_start = 1 / n_hidden_states
        alpha_ = np.zeros((n_hidden_states, len(observations)), dtype=float)
        #TODO complete the forward function to calculate alpha
        n_transit = len(observations)
        
        # Init
        for i in range(n_hidden_states):
            alpha_[i][0] = a_start * b_[i][observations[0]]

        # Induction
        for t in range(n_transit-1):
            for j in range(n_hidden_states):
                alpha_[j][t+1] = np.sum(np.array([alpha_[i][t] * a_[i][j] for i in range(n_hidden_states)])) * b_[j][observations[t+1]]

        return alpha_

    def backward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        backward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined beta_
        """
        beta_ = np.zeros((n_hidden_states, len(observations)), dtype=float)
        beta_[:, -1:] = 1
        # TODO finish the function to calculate backward pass and calculate beta
        n_transit = len(observations)

        # Induction
        for t in np.arange(n_transit-2, -1, -1):
            for j in range(n_hidden_states):
                beta_[j][t] = np.sum([a_[j][i] * b[i][observations[t+1]] * beta_[i][t+1] for i in range(n_hidden_states)])

        return beta_

    def compute_gamma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        # gamma_prob = np.zeros(n_samples, len(observations))
        gamma_prob = np.multiply(alfa, beta) / sum(np.multiply(alfa, beta))
        return gamma_prob

    def compute_sigma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        sigma_prob = np.zeros((n_samples, len(observations) - 1, n_samples), dtype=float)
        denomenator = np.multiply(alfa, beta)
        for i in range(len(observations) - 1):
            for j in range(n_samples):
                for k in range(n_samples):
                    index_in_vocab = np.where(vocab == observations[i + 1])[0][0]
                    sigma_prob[j, i, k] = (alfa[j, i] * beta[k, i + 1] * a_[j, k] * b_[k, index_in_vocab]) / sum(
                        denomenator[:, j])
        return sigma_prob

    # initialize A ,B
    a = np.ones((n_hidden_states, n_hidden_states)) / n_hidden_states
    b = np.ones((n_hidden_states, len(observations_vocab))) / len(observations_vocab)
    for iter in tqdm(range(2000), position=0, leave=True):

        # E-step caclculating sigma and gamma
        alfa_prob = forward_probs(observations, observations_vocab, n_hidden_states, a, b)  #
        beta_prob = backward_probs(observations, observations_vocab, n_hidden_states, a, b)  # , beta_val
        gamma_prob = compute_gamma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)
        sigma_prob = compute_sigma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)

        # M-step caclculating A, B matrices
        a_model = np.zeros((n_hidden_states, n_hidden_states))
        for j in range(n_hidden_states):  # calculate A-model
            for i in range(n_hidden_states):
                for t in range(len(observations) - 1):
                    a_model[j, i] = a_model[j, i] + sigma_prob[j, t, i]
                normalize_a = [sigma_prob[j, t_current, i_current] for t_current in range(len(observations) - 1) for
                               i_current in range(n_hidden_states)]
                normalize_a = sum(normalize_a)
                if normalize_a == 0:
                    a_model[j, i] = 0
                else:
                    a_model[j, i] = a_model[j, i] / normalize_a

        b_model = np.zeros((n_hidden_states, len(observations_vocab)))

        for j in range(n_hidden_states):
            for i in range(len(observations_vocab)):
                indices = [idx for idx, val in enumerate(observations) if val == observations_vocab[i]]
                numerator_b = sum(gamma_prob[j, indices])
                denominator_b = sum(gamma_prob[j, :])
                if denominator_b == 0:
                    b_model[j, i] = 0
                else:
                    b_model[j, i] = numerator_b / denominator_b

        a = a_model
        b = b_model
    return a, b


import random
random.seed(40)
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observable_map = {'sleeping': 0, 'eating': 1, 'pooping': 2}
observations = []
for i in range(100):
    observations.append(observable_map[random.choice(observable_states)])
# print(observations)
# print(np.array(list(observable_map.values())))

A, B = baum_welch(observations=observations, observations_vocab=np.array(list(observable_map.values())),
                  n_hidden_states=2)


100%|██████████| 2000/2000 [00:12<00:00, 160.88it/s]


In [67]:
print(A)
print(B)

[[0.5 0.5]
 [0.5 0.5]]
[[0.36 0.31 0.33]
 [0.36 0.31 0.33]]


In [68]:
import random
random.seed(42)

hidden_states = ['hot', 'mild', 'cold']
observable_states = ['dress', 't-shirt', 'coat']

# make dataset
N = 100
states = []
observations = []
for i in range(N):
    states.append(random.choice(hidden_states))

for s in states:
    rnd = random.random()
    if s == 'hot':
        if rnd < 0.60:
            observations.append(random.choice(['t-shirt']))
        else:
            observations.append(random.choice(['dress']))
    elif s == 'mild':
        if rnd < 0.60:
            observations.append(random.choice(['dress', 't-shirt']))
        else:
            observations.append(random.choice(['coat']))
    else:
        if rnd < 0.10:
            observations.append(random.choice(['dress', 't-shirt']))
        else:
            observations.append(random.choice(['coat']))


In [69]:
#TASK 4: Now try it with your HMM
observations = [obs2id[obs] for obs in observations]
A, B = baum_welch(observations=observations, observations_vocab=np.arange(0, len(observable_states)), n_hidden_states=len(hidden_states))

100%|██████████| 2000/2000 [00:25<00:00, 78.24it/s]


In [70]:
print(A)
print(B)

[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]
[[0.27 0.35 0.38]
 [0.27 0.35 0.38]
 [0.27 0.35 0.38]]
